In [ ]:
import streamlit as st
import folium
from folium.plugins import HeatMap, MarkerCluster
import pandas as pd
import numpy as np
import tempfile
import os
import json

def load_data():
    df = pd.read_csv("data.csv")
    df["Начало"] = pd.to_datetime(df["Начало"], errors='coerce')
    df["Конец"]  = pd.to_datetime(df["Конец"],  errors='coerce')
    return df

df = load_data()

def calculate_time_spent(df_local, threshold=1e-4):
    df_copy = df_local.copy()
    df_copy.sort_values(by=["Группировка", "Начало"], inplace=True)
    df_copy["dwelling_time"] = 0.0

    for group_name, group_data in df_copy.groupby("Группировка", group_keys=False):
        idx_list = group_data.index.to_list()
        for i in range(len(idx_list) - 1):
            idx_i  = idx_list[i]
            idx_i1 = idx_list[i + 1]
            
            lat_end_i    = group_data.loc[idx_i, "latitude_конеч"]
            lon_end_i    = group_data.loc[idx_i, "longitude_конеч"]
            lat_start_i1 = group_data.loc[idx_i1, "latitude_нач"]
            lon_start_i1 = group_data.loc[idx_i1, "longitude_нач"]
            
            dist = np.sqrt((lat_end_i - lat_start_i1)**2 + (lon_end_i - lon_start_i1)**2)
            if dist < threshold:
                t_end   = group_data.loc[idx_i, "Конец"]
                t_start = group_data.loc[idx_i1, "Начало"]
                if pd.notnull(t_end) and pd.notnull(t_start):
                    delta_sec = (t_start - t_end).total_seconds()
                    if delta_sec > 0:
                        df_copy.at[idx_i, "dwelling_time"] = delta_sec
    return df_copy

df_time = calculate_time_spent(df, threshold=1e-4)

# Для HeatMap агрегируем по координатам конечных точек
df_time_sum = df_time.groupby(
    ["latitude_конеч", "longitude_конеч"], dropna=False
)["dwelling_time"].sum().reset_index()


# 2) Слой с HeatMap (по времени пребывания)
heat_data = []
for _, row in df_time_sum.iterrows():
    lat = row["latitude_конеч"]
    lon = row["longitude_конеч"]
    weight = row["dwelling_time"]
    if pd.notnull(lat) and pd.notnull(lon):
        heat_data.append([lat, lon, weight])


# Создаем DataFrame только с событиями, где dwelling_time > 0
df_dwell_events = df_time[df_time["dwelling_time"] > 0].copy()
df_dwell_events["Прибытие"] = df_dwell_events["Конец"] + pd.to_timedelta(df_dwell_events["dwelling_time"], unit="s")
df_dwell_events["Конец_str"] = df_dwell_events["Конец"].dt.strftime("%Y-%m-%d %H:%M:%S")
df_dwell_events["Прибытие_str"] = df_dwell_events["Прибытие"].dt.strftime("%Y-%m-%d %H:%M:%S")

# Группируем события по конечным координатам
grouped_events = df_dwell_events.groupby(["latitude_конеч", "longitude_конеч"])
popup_texts = {}
agent_tooltips = {}
for (lat, lon), group in grouped_events:
    # Собираем уникальные имена агентов в этой точке
    agents = group["Группировка"].unique()
    agent_names = ", ".join(agents)
    lines = [f"Агент: {agent_names}"]
    for _, row in group.iterrows():
        lines.append(f"Прибытие: {row['Конец_str']}<br>Отъезд: {row['Прибытие_str']}")
    popup_text = "<hr>".join(lines)
    popup_texts[(lat, lon)] = popup_text
    agent_tooltips[(lat, lon)] = agent_names


In [8]:
df_time_sum

,latitude_конеч,longitude_конеч,dwelling_time
0,48.743520,82.692510,2733.0
1,48.754310,82.384610,1412.0
2,48.755350,82.379780,632.0
3,48.843560,82.747510,1893.0
4,48.869510,83.254280,334.0
...,...,...,...
4792,54.944400,69.197570,1112.0
4793,54.944493,69.180794,89449.0
4794,54.953190,69.243690,2732.0
4795,54.962000,69.037540,1712.0


In [ ]:
df_2 = pd.DataFrame(heat_data)


,0,1,2
0,48.743520,82.692510,2733.0
1,48.754310,82.384610,1412.0
2,48.755350,82.379780,632.0
3,48.843560,82.747510,1893.0
4,48.869510,83.254280,334.0
...,...,...,...
4792,54.944400,69.197570,1112.0
4793,54.944493,69.180794,89449.0
4794,54.953190,69.243690,2732.0
4795,54.962000,69.037540,1712.0
